In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install libraries

In [ ]:
! pip install --quiet langchain langchain_cohere gpt4all langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/GAN_class/FirstProject

/content/drive/MyDrive/GAN_class/FirstProject


## Create agents

In [ ]:
import os
from langchain_cohere.chat_models import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain.agents import AgentExecutor

from Tool_DataBase import flights_functions, hotel_functions, database_create
from Tool_FAQ import policy_functions
import Tool_Search
from Tool_Search import search_functions


os.environ['COHERE_API_KEY'] = ""
llm_model = ChatCohere(model="command-r-plus", temperature=0.3)


# Create the prompt
prompt = ChatPromptTemplate.from_template("{input} Just search in Persian-language in SQLite for every response.")

my_tools = [search_functions.internet_search,
            policy_functions.lookup_policy,
            flights_functions.search_flights,
            flights_functions.take_ticket,
            flights_functions.cancel_ticket,
            hotel_functions.cancel_hotel,
            hotel_functions.book_hotel,
            hotel_functions.search_hotel]

# Create the ReAct agent
# Create an agent that enables multiple tools to be used in sequence to complete a task.
agent = create_cohere_react_agent(llm = llm_model,
                                  tools = my_tools,
                                  prompt=prompt,)

# Agent that is using tools.
agent_executor = AgentExecutor(agent = agent,
                               tools = my_tools,
                               verbose = True)



questions = {
    "input3":"آیا ساعت، قیمت و دیگر مشخصات پروازها روی سایت علی‌بابا قطعی است؟",
    "input1": "می خوام از اصفهان برم مشهد برای چه ساعت هایی داره؟",
    "input2":"برای پرواز شماره 3 یه بلیط می خوام رزرو کن لطفا" + "passenger_id=2",
    "input4": "هتل های شیراز کجاها هستند؟",
    "input5": "برای من هتل 7 رو رزرو کن " +"passenger_id=1",
    "input6": "هتل 1 رو برای من کنسل کن نمی خوام" + "passenger_id=1",
    "input7":"زحمتی بکش بلیط پرواز رو برای من لفو کن."+ "passenger_id=2"+ "ticket_id=3"
    }

for key, value in questions.items():
    # Invoke the agent with each question
    result = agent_executor.invoke({
        "input": value
    })


SQL script executed successfully.


> Entering new AgentExecutor chain...

I will search for the answer in the database.
{'tool_name': 'lookup_policy', 'parameters': {'query': 'آیا ساعت، قیمت و دیگر مشخصات پروازها روی سایت علییبابا قطعĠی است؟'}}
بله، آنچه پس از جستجوی پرواز مورد نظر خود روی سایت علی‌بابا می‌بینید، برنامه قطعی پروازهای ایرلاین‌هاست که به ریال محاسبه شده است. اگر احیانا تغییری در ساعت یا برنامه‌ی پرواز به وجود بیاید، از طریق پشتیبانی سایت علی‌بابا به اطلاع شما خواهد رسید.Relevant Documents: 0
Cited Documents: 0
Answer: بله، آنچه پس از جستجوی پرواز مورد نظر خود روی سایت علی‌بابا می‌بینید، برنامه قطعی پروازهای ایرلاین‌هاست که به ریال محاسبه شده است. اگر احیانا تغییری در ساعت یا برنامه‌ی پرواز به وجود بیاید، از طریق پشتیبانی سایت علی‌بابا به اطلاع شما خواهد رسید.
Grounded answer: <co: 0>بله، آنچه پس از جستجوی پرواز مورد نظر خود روی سایت علی‌بابا می‌بینید، برنامه قطعی پروازهای ایرلاین‌هاست که به ریال محاسبه شده است. اگر احیانا تغییری در ساعت یا برنامه‌ی پرواز به وجود بیاید، 

## Check tabel name in my database

In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/GAN_class/FirstProject/my_database.db')

cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()
print(tables)

conn.close()


[('sqlite_sequence',), ('destination',), ('hotel',), ('hotel_book',), ('info_flight',), ('passenger',), ('ticket_airplane',)]


## Get all data a tabel

In [ ]:
import sqlite3
conn = sqlite3.connect('/content/drive/MyDrive/GAN_class/FirstProject/my_database.db')
cursor = conn.cursor()
params = []
table_name = "passenger"

query = "SELECT * FROM "+ table_name
cursor.execute(query)
rows = cursor.fetchall()
column_names = [column[0] for column in cursor.description]
results = [dict(zip(column_names, row)) for row in rows]

cursor.close()
conn.close()
print(results)

[{'id': 1, 'name': 'ساناز', 'family': 'خلیلی', 'city': 'رفسنجان'}]


## Get id hotel

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/GAN_class/FirstProject/my_database.db')
cursor = conn.cursor()
query = "SELECT id from hotel WHERE name=? ;"
cursor.execute(query, ('پردیسان',))
hotel_id = cursor.fetchone()
hotel_id

(7,)

## Add user a passegenr

In [ ]:
import sqlite3
name = "حسن"
family = "حسینی"
city = "کرمان"
conn = sqlite3.connect('/content/drive/MyDrive/GAN_class/FirstProject/my_database.db')
cursor = conn.cursor()

insert_query = "INSERT INTO passenger (name, family, city) VALUES (?, ?, ?);"
cursor.execute(insert_query, (name, family, city))

conn.commit()


cursor.close()
conn.close()